In [ ]:
import pandas as pd
import numpy as np
tb_veic = pd.read_csv('data/dados_veiculos.csv')

In [ ]:
tb_veic.columns = [column.lower().replace(' ', '_').replace('/', '_by_') for column in tb_veic.columns]

In [ ]:
tb_veic.info()

In [ ]:
for column in tb_veic.columns:
    print(f'{column}: {len(tb_veic[column].unique())}')

# Data Manipulation - Group By

- `.groupby()` é uma forma de **agregar** todos os resultados para cada chave única
- Sempre que você faz uma **agregação**, o resultado final terá 1 linha para cada valor pelo qual você agregou, portanto, é obrigatório que se aplique uma função agregadora para que todos os valores sejam sumarizados em um único valor associado àquela chave.

Por exemplo, se tivermos:

Fruta | Quantidade
-----|----:
Laranja | 10
Maçã | 10
Abacate | 2
Abacate | 1
Melancia  | 1
Laranja  | 4

O resultado de um `.groupby` por 'Fruta' resultaria em 4 linhas

Fruta | Função Agregadora
----|-----
Laranja | ?
Maçã | ?
Abacate | ?
Melancia | ?

O asterisco representa o valor agregado. Temos, obrigatoriamente, que sumarizar os dados relacionados àquele registro em um único dado. Para isso, podemos fazer a média, soma, contagem, ou qualquer outra função agregadora.

Fruta | soma(Quantidade)
----|----:
Laranja | 14
Maçã | 10
Abacate | 3
Melancia | 1

**Funções agregadora/Métodos**

- `.mean()`
- `.median()`
- `.max()`
- `.min()`
- `.sum()`
- `.count()`
- `.describe()`
- `.agg()`
- A function you want

## One aggregation

Key with one value and one aggregation method

In [ ]:
tb_veic.groupby(by = 'drivetrain').mean()

In [ ]:
tb_veic.groupby(by = 'drivetrain').median()

In [ ]:
tb_agg_drivetrain = tb_veic.groupby(by = 'drivetrain').median()
type(tb_agg_drivetrain)

In [ ]:
tb_agg_drivetrain.loc['4-Wheel or All-Wheel Drive',:]

## More than one aggregation

Key with one or more values and one or more aggregation methods

In [ ]:
tb_agg_cyldt = tb_veic.groupby(by = ['drivetrain', 'cylinders']).median()

In [ ]:
tb_agg_cyldt

In [ ]:
tb_agg_cyldt.loc[('Rear-Wheel Drive',6),:]

In [ ]:
tb_agg_cyldt.loc[('Rear-Wheel Drive',[2,3,4]),:]

In [ ]:
tb_agg_cyldt.loc[('Rear-Wheel Drive',[2,3,4]),'city_mpg']

In [ ]:
tb_agg_cyldt.loc[('Rear-Wheel Drive',[2,3,4]),['city_mpg', 'highway_mpg']]

In [ ]:
tb_agg_cyldt.reset_index()

In [ ]:
tb_agg_desc_cyldt = tb_veic.groupby(by = ['drivetrain', 'cylinders']).describe()
tb_agg_desc_cyldt

In [ ]:
tb_agg_desc_cyldt.loc[('Rear-Wheel Drive', 6),('city_mpg', 'mean')]

In [ ]:
tb_agg_desc_cyldt.loc[('Rear-Wheel Drive', [2,3,4]),('city_mpg', 'mean')]

In [ ]:
tb_agg_desc_cyldt.loc[('Rear-Wheel Drive', [2,3,4]),('city_mpg', ['mean', 'std'])]

In [ ]:
tb_agg_desc_cyldt.loc[('Rear-Wheel Drive', ),('city_mpg', )]

In [ ]:
tb_agg_musig_cyldt = tb_veic.groupby(by = ['drivetrain', 'cylinders']).agg(['mean', 'std'])
tb_agg_musig_cyldt

## Named aggregation

In [ ]:
tb_veic.groupby(by = ['drivetrain', 'cylinders']).agg(contagem = pd.NamedAgg(column = 'year', aggfunc = 'count'))

In [ ]:
tb_veic.groupby(by = ['drivetrain', 'cylinders']).agg(
    contagem = pd.NamedAgg(column = 'year', aggfunc = 'count'),
    avg_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'mean'),
    std_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'std'),
    avg_highway_mpg = pd.NamedAgg(column = 'highway_mpg', aggfunc = 'mean'),
    std_highway_mpg = pd.NamedAgg(column = 'highway_mpg', aggfunc = 'std'))

In [ ]:
tb_agg_veic = tb_veic.groupby(by = ['drivetrain', 'cylinders']).agg(
    contagem = pd.NamedAgg(column = 'year', aggfunc = 'count'),
    avg_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'mean'),
    std_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'std'),
    avg_highway_mpg = pd.NamedAgg(column = 'highway_mpg', aggfunc = 'mean'),
    std_highway_mpg = pd.NamedAgg(column = 'highway_mpg', aggfunc = 'std'))
tb_agg_veic[tb_agg_veic['contagem'] > 5]

In [ ]:
def dist_iq(panda_series):
    if len(panda_series) > 5:
        return panda_series.quantile(0.75) - panda_series.quantile(0.25)
    else:
        return np.nan

In [ ]:
tb_veic.groupby(by = ['drivetrain', 'cylinders']).agg(
    avg_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'mean'),
    std_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'std'),
    iqn_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = dist_iq))

## Bonus: Iterating over groups

In [ ]:
tb_gr_veic = tb_veic.groupby(by = ['drivetrain'])
for name, group in tb_gr_veic:
    print(name)
    print(group.describe())

-----

# Data Manipulation - Joins

* How to merge dataframes based on one or more keys
* Diagramas de Venn
* [Fruits Example](https://docs.google.com/spreadsheets/d/1foV1THfhVc5WduBTMjpxmthAD1BDHS9FUpH-WgGDH5E/edit?usp=sharing)

Um join pode ser classificado ao longo de dois eixos:

1. **Direção**: Esquerda, direita ou interno (left, right e inner);
2. **Cardinalidade**: Um para um, um para muitos, muitos para muitos (1:1, 1:n, n:n).

## Types of Joins

### Um para Um (1:1)

In [ ]:
nome_frutas = ['Uva', 'Abacate', 'Melancia', 'Atemoia', 'Laranja']
producao_frutas = [10, 1, 1, 3, 4]
tb_prod_frutas = pd.DataFrame({'nome_fruta' : nome_frutas,
                               'producao_kg' : producao_frutas})
tb_prod_frutas

In [ ]:
fruta = ['Uva', 'Abacate', 'Melancia', 'Carambola']
cidade = ['Campinas', 'Campinas', 'Atibaia', 'Rio de Janeiro']
tb_pomar_cidade = pd.DataFrame({'nome_fruta' : fruta, 'cidade' : cidade})
tb_pomar_cidade

#### Inner Join (o que é comum às duas tabelas)

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on = 'nome_fruta')

#### Left Join (tudo o que está na tabela a esquerda)

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on = 'nome_fruta', how = 'left')

#### Right Join (tudo o que está na tabela a direita)

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on = 'nome_fruta', how = 'right')

### Um para N (1:N)

In [ ]:
nome_frutas = ['Uva', 'Abacate', 'Melancia', 'Atemoia', 'Laranja']
producao_frutas = [10, 1, 1, 3, 4]
tb_prod_frutas = pd.DataFrame({'nome_fruta' : nome_frutas,
                               'producao_kg' : producao_frutas})
tb_prod_frutas

In [ ]:
fruta = ['Uva', 'Abacate', 'Melancia', 'Uva', 'Carambola']
cidade = ['Campinas', 'Campinas', 'Atibaia', 'Atibaia', 'Rio de Janeiro']
tb_pomar_cidade = pd.DataFrame({'nome_fruta' : fruta, 'cidade' : cidade})
tb_pomar_cidade

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on = 'nome_fruta')

### N para N (N:N)

In [ ]:
nome_frutas = ['Uva', 'Uva', 'Abacate', 'Melancia', 'Atemoia', 'Laranja']
producao_frutas = [10, 5, 1, 1, 3, 4]
tb_prod_frutas = pd.DataFrame({'nome_fruta' : nome_frutas,
                               'producao_kg' : producao_frutas})
tb_prod_frutas

In [ ]:
fruta = ['Uva', 'Abacate', 'Melancia', 'Uva', 'Carambola']
cidade = ['Campinas', 'Campinas', 'Atibaia', 'Atibaia', 'Rio de Janeiro']
tb_pomar_cidade = pd.DataFrame({'nome_fruta' : fruta, 'cidade' : cidade})
tb_pomar_cidade

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, on = 'nome_fruta')

## Using our vehicles dataframe

Será que a eficiência dos carros aumentou ao longo dos anos?

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tb_agg_year = tb_veic.groupby(by = 'year').agg(
    mean_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'mean'),
    mean_highway_mpg = pd.NamedAgg(column = 'highway_mpg', aggfunc = 'mean')
)
plt.plot(tb_agg_year['mean_city_mpg']);

**Quais conclusões podemos tirar do gráfico acima? Como podemos melhorar nossa análise?**

In [ ]:
tb_agg_cyl = tb_veic.groupby(by = 'cylinders').agg(
    mean_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'mean'),
    mean_highway_mpg = pd.NamedAgg(column = 'highway_mpg', aggfunc = 'mean')
).reset_index()
tb_agg_cyl

In [ ]:
tb_veic = pd.merge(tb_veic, tb_agg_cyl, on = 'cylinders')
tb_veic['norm_city_mpg'] = tb_veic['city_mpg']/tb_veic['mean_city_mpg']
tb_veic['norm_highway_mpg'] = tb_veic['highway_mpg']/tb_veic['mean_highway_mpg']

In [ ]:
tb_agg_year = tb_veic.groupby(by = 'year').agg(
    mean_city_mpg = pd.NamedAgg(column = 'norm_city_mpg', aggfunc = 'mean'),
    mean_highway_mpg = pd.NamedAgg(column = 'norm_highway_mpg', aggfunc = 'mean')
)
plt.plot(tb_agg_year['mean_city_mpg'])

## What if we had different names?

In [ ]:
nome_frutas = ['Uva', 'Abacate', 'Melancia', 'Atemoia', 'Laranja']
producao_frutas = [10, 1, 1, 3, 4]
tb_prod_frutas = pd.DataFrame({'nome_fruta' : nome_frutas,
                               'producao_kg' : producao_frutas})
tb_prod_frutas

In [ ]:
fruta = ['Uva', 'Abacate', 'Melancia', 'Carambola']
cidade = ['Campinas', 'Campinas', 'Atibaia', 'Rio de Janeiro']
tb_pomar_cidade = pd.DataFrame({'nome_pomar_fruta' : fruta, 'cidade' : cidade})
tb_pomar_cidade

In [ ]:
pd.merge(tb_prod_frutas, tb_pomar_cidade, left_on = 'nome_fruta', right_on = 'nome_pomar_fruta')

In [ ]:
tb_agg_yearcyl = tb_veic.groupby(['cylinders', 'year']).agg(
    mean_city_mpg = pd.NamedAgg(column = 'city_mpg', aggfunc = 'mean'),
    n_linhas = pd.NamedAgg(column = 'city_mpg', aggfunc = 'count')
).reset_index()
tb_agg_yearcyl

In [ ]:
tb_agg_yearcyl['last_year'] = tb_agg_yearcyl['year'] - 1
tb_agg_yearcyl

In [ ]:
tb_agg_yearcyla = pd.merge(tb_agg_yearcyl, tb_agg_yearcyl, 
                           left_on = ['cylinders', 'last_year'], 
                           right_on = ['cylinders', 'year'],
                           suffixes = ('', '_ly'))
tb_agg_yearcyla

In [ ]:
tb_agg_yearcyla['var_anual'] = (tb_agg_yearcyla['mean_city_mpg'] - tb_agg_yearcyla['mean_city_mpg_ly'])/tb_agg_yearcyla['mean_city_mpg_ly']

In [ ]:
tb_agg_yearcyla.groupby(['cylinders']).agg(
    var_anual_media = pd.NamedAgg('var_anual', 'mean'),
    n_linhas = pd.NamedAgg('n_linhas', 'sum')
)

# Bonus - Bins Concept

![Imgs](https://i.stack.imgur.com/pObHa.png)


### two methods: cut vs qcut

The major distinction is that qcut will calculate the size of each bin in order to make sure the distribution of data in the bins is equal. In other words, all bins will have (roughly) the same number of observations but the bin range will vary.

On the other hand, cut is used to specifically define the bin edges. There is no guarantee about the distribution of items in each bin. In fact, you can define bins in such a way that no items are included in a bin or nearly all items are in a single bin.

In [ ]:
pd.cut(tb_veic['city_mpg'], bins = 10)

In [ ]:
tb_veic['classifA_city_mpg'] = pd.cut(tb_veic['city_mpg'], bins = 10)
tb_veic['classifA_city_mpg'].value_counts()

In [ ]:
tb_veic['classifB_city_mpg'] = pd.qcut(tb_veic['city_mpg'], q = 10)
tb_veic['classifB_city_mpg'].value_counts()

# Bonus - Categorical Data Into Columns

>    - dummies
>    - One hot encoding

In [ ]:
pd.get_dummies(tb_veic['classifB_city_mpg'])

In [ ]:
dummies = pd.get_dummies(tb_veic['classifB_city_mpg'])
pd.concat([tb_veic, dummies], axis = 1)